In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.ticker as ticker
from google.cloud import bigquery
import os

!pip install openpyxl ## read excel files
!pip install wrds
import wrds

print('----------------\n Establishing WRDS connection:')
# establish connection
conn = wrds.Connection()



----------------
 Establishing WRDS connection:


Enter your WRDS username [jupyter]: rogermasclans
Enter your password: ········


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  y


Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [4]:


stock_index_data = conn.raw_sql(
f"""

SELECT
    comp_global_daily.g_names_ix.tic,
    comp_global_daily.g_idx_daily.datadate,
    comp_global_daily.g_names_ix.gvkeyx,
    comp_global_daily.g_names_ix.conm,
    comp_global_daily.g_names_ix.indextype,
    comp_global_daily.g_names_ix.indexid,
    comp_global_daily.g_names_ix.indexcat,
    comp_global_daily.g_names_ix.idxiddesc,
    comp_global_daily.g_idx_daily.dvpsxd,
    comp_global_daily.g_idx_daily.newnum,
    comp_global_daily.g_idx_daily.oldnum,
    comp_global_daily.g_idx_daily.prccd,
    comp_global_daily.g_idx_daily.prccddiv,
    comp_global_daily.g_idx_daily.prccddivn,
    comp_global_daily.g_idx_daily.prchd,
    comp_global_daily.g_idx_daily.prcld

FROM comp_global_daily.g_idx_daily
LEFT JOIN comp_global_daily.g_names_ix
    ON comp_global_daily.g_idx_daily.gvkeyx = comp_global_daily.g_names_ix.gvkeyx

WHERE comp_global_daily.g_idx_daily.datadate BETWEEN
    '1980-01-01'::date AND '1999-12-31'::date"""
)
stock_index_data







,tic,datadate,gvkeyx,conm,indextype,indexid,indexcat,idxiddesc,dvpsxd,newnum,oldnum,prccd,prccddiv,prccddivn,prchd,prcld
0,I1JOR001,2000-01-02,150195,Amman Stock Exchange Price Index,COMPOSITE,JOR,EXCHG,Amman Stock Exchange,None,None,None,1673.50,None,None,None,None
1,I1ISR002,2000-01-02,150346,Tel Aviv 25 Index,COMPOSITE,TELAVIV,EXCHG,Tel Aviv Stock Exchange,None,None,None,484.64,None,None,None,None
2,I1ISR001,2000-01-02,150925,Tel Aviv 100 Index,COMPOSITE,TELAVIV,EXCHG,Tel Aviv Stock Exchange,None,None,None,487.23,None,None,None,None
3,I6UNK158,2000-01-02,151069,FTSE Eurofirst 300 Autos & Parts Index,SECTOR,FTSEU300,FTSEU,FTSE Eurofirst 300,None,None,None,863.65,None,None,None,None
4,I1EGY001,2000-01-02,151107,Hermes Egypt Stock Market,COMPOSITE,EGY,EXCHG,Hermes Egypt Stock Market,None,None,None,130.28,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281815,I3GBR114,2009-12-31,277873,FTSE AIM All-Share Technology Index,SECTOR,AIM,FT,AIM Index,None,None,None,3700.62,None,None,None,None
281816,I3GBR115,2009-12-31,277874,FTSE AIM All-Share Telecommunications Index,SECTOR,AIM,FT,AIM Index,None,None,None,1889.40,None,None,None,None
281817,I3GBR116,2009-12-31,277875,FTSE AIM All-Share Travel & Leisure Index,SECTOR,AIM,FT,AIM Index,None,None,None,1771.56,None,None,None,None
281818,I3GBR117,2009-12-31,277876,FTSE AIM All-Share Utilities Index,SECTOR,AIM,FT,AIM Index,None,None,None,6098.83,None,None,None,None


In [ ]:

####SET BG PROJECT TO FETCH DATA
# Define project on BQ
project_id = "starfire-410116"

# Authenticate to Google Cloud
client = bigquery.Client(project=project_id)

# Set allow_large_results to Truepoti
job_config = bigquery.QueryJobConfig()
job_config.allow_large_results = True

dataset_id = 'derived'
table_id = 'stock_index_data_2000_2009'

# Authenticate with Google Cloud using your service account key
client = bigquery.Client(project=project_id)

# Specify the BigQuery table reference
table_ref = client.dataset(dataset_id).table(table_id)

# Upload the Pandas DataFrame to BigQuery with auto-detection
job_config = bigquery.LoadJobConfig(
  autodetect=True, # Enable auto-detection of the schema
  write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE # You can choose your write disposition
)

# Upload the DataFrame to BigQuery
job = client.load_table_from_dataframe(stock_index_data, table_ref, job_config=job_config)
job.result() # Wait for the job to complete
print(f'... done')
